## There are three ways to run BertSum  
- Use the Entirely pretrained Model. You can directly run your test on this Model without training
- Use the Bert pretrained model and add fine tuning on top of it for summarization.. You need to train this Model
- Use Transformer summarization code : This is similar concept as pt 2 above, but the code is different. I will run it to see if it gives a different Rouge score

In [7]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os, sys, re, json, time, datetime, shutil
import nltk
import sentencepiece as sp
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from transformers import *
import torch

In [4]:

# Check versions
print(np.__version__)
print(pd.__version__)
print(tf.__version__)
print(torch.__version__)

1.16.2
0.24.2
1.14.0
1.3.0.post2


## Load CNN/DailyMail dataset

BertSum team has already preprocessed the data and split it into train,test and validation set in the .pt format.The .pt files are located in PreSumm/bert_data .

## CNN_DM abstractive from Pretrained Model

The BertSum pretrained model is downloaded and stored in PreSumm/pretrained folder. model_step_148000.pt is the abstractive pretrained model and bertext_cnndm_transformer.pt is the extractive pretained model. You can directly evaluate or test the model - no need to train it.

In [3]:
%cd PreSumm/src

/Users/adas1/Aditi/personal/school/266/project/PreSumm/src


### Run the model for ~1K dataset along with the Rouge score

In [4]:
!python train.py -task abs -mode test -test_from ../pretrained/model_step_148000.pt -batch_size 32 -test_batch_size 500 -bert_data_path ../small_data/cnndm -log_file ../logs/pretrained_abs_bert_cnndm -report_rouge True  -sep_optim true -use_interval true -visible_gpus -1 -max_pos 512 -max_length 200 -alpha 0.95 -min_length 50 -result_path ../results/pretrained_abs_bert_cnndm_sample

[2019-11-09 08:59:13,583 INFO] Loading checkpoint from ../pretrained/model_step_148000.pt
Namespace(accum_count=1, alpha=0.95, batch_size=32, beam_size=5, bert_data_path='../small_data/cnndm', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0], label_smoothing=0.1, large=False, load_from_extractive='', log_file='../logs/pretrained_abs_bert_cnndm', lr=1, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=200, max_pos=512, max_tgt_len=140, min_length=50, mode='test', model_path='../models/', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=True, result_path='../results/pretrained_abs_bert_cnndm_sample', save_checkpoint_steps=5, seed=6

[2019-11-09 11:02:37,474 INFO] Rouges at step 148000 
>> ROUGE-F(1/2/3/l): 40.67/18.52/37.69
ROUGE-R(1/2/3/l): 43.08/19.56/39.89



## CNN_DM extractive from Pretrained Model

In [3]:
%cd PreSumm/src
!python train.py -task ext -mode test -test_from ../pretrained/bertext_cnndm_transformer.pt -batch_size 32 -test_batch_size 500 -bert_data_path ../small_data/cnndm -log_file ../logs/pretrained_ext_bert_cnndm -report_rouge True  -sep_optim true -use_interval true -visible_gpus -1 -max_pos 512 -max_length 200 -alpha 0.95 -min_length 50 -result_path ../results/pretrained_ext_bert_cnndm_sample

/Users/adas1/Aditi/personal/school/266/project/PreSumm/src
[2019-11-09 11:26:00,754 INFO] Loading checkpoint from ../pretrained/bertext_cnndm_transformer.pt
Namespace(accum_count=1, alpha=0.95, batch_size=32, beam_size=5, bert_data_path='../small_data/cnndm', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0], label_smoothing=0.1, large=False, load_from_extractive='', log_file='../logs/pretrained_ext_bert_cnndm', lr=1, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=200, max_pos=512, max_tgt_len=140, min_length=50, mode='test', model_path='../models/', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=True, result_path='../results

## CNN_DM extractive from Pretrained BERT but finetuned for BertSUM
Here you have to train the model and then evaluate the model

In [4]:
%cd PreSumm/src
!python train.py -task ext -mode train -bert_data_path ../small_data/cnndm -ext_dropout 0.1 -model_path ../models/bert_classifier -lr 2e-3 -visible_gpus -1 -report_every 50 -save_checkpoint_steps 500 -batch_size 30 -train_steps 1000 -accum_count 2 -log_file ../logs/ext_bert_cnndm -use_interval true -warmup_steps 10 -max_pos 512

[Errno 2] No such file or directory: 'PreSumm/src'
/Users/adas1/Aditi/personal/school/266/project/PreSumm/src
[2019-11-09 12:14:13,870 INFO] Device ID -1
[2019-11-09 12:14:13,870 INFO] Device cpu
[2019-11-09 12:14:14,321 INFO] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at ../temp/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
[2019-11-09 12:14:14,323 INFO] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 

[2019-11-09 12:14:16,421 INFO] * number of parameters: 120512513
[2019-11-09 12:14:16,421 INFO] Start training...
[2019-11-09 12:14:16,530 INFO] Loading train dataset from ../small_data/cnndm.train.143.bert.pt, number of examples: 1084
[2019-11-09 12:17:27,118 INFO] Step 50/ 1000; xent: 4.41; lr: 0.0002828;   1 docs/s;    191 sec
[2019-11-09 12:20:56,063 INFO] Step 100/ 1000; xent: 3.23; lr: 0.0002000;   1 docs/s;    400 sec
[2019-11-09 12:24:15,563 INFO] Step 150/ 1000; xent: 3.13; lr: 0.0001633;   1 docs/s;    599 sec
[2019-11-09 12:27:35,911 INFO] Step 200/ 1000; xent: 3.15; lr: 0.0001414;   1 docs/s;    799 sec
[2019-11-09 12:31:20,790 INFO] Step 250/ 1000; xent: 3.12; lr: 0.0001265;   1 docs/s;   1024 sec
[2019-11-09 12:34:55,520 INFO] Step 300/ 1000; xent: 3.11; lr: 0.0001155;   1 docs/s;   1239 sec
[2019-11-09 12:38:34,061 INFO] Step 350/ 1000; xent: 3.34; lr: 0.0001069;   1 docs/s;   1458 sec
[2019-11-09 12:42:16,126 INFO] Step 400/ 1000; xent: 3.26; lr: 0.0001000;   1 docs/s; 

### Validate with Rouge score

In [6]:
%cd PreSumm/src
!python train.py -task ext -mode validate -batch_size 30 -test_batch_size 50 -bert_data_path ../small_data/cnndm -log_file ../logs/ext_val_bert_cnndm -model_path ../models/bert_classifier -sep_optim true -use_interval true -visible_gpus -1 -max_pos 512 -max_length 200 -alpha 0.95 -min_length 50 -result_path ../logs/ext_val_bert_cnndm 

[Errno 2] No such file or directory: 'PreSumm/src'
/Users/adas1/Aditi/personal/school/266/project/PreSumm/src
[2019-11-09 14:36:05,500 INFO] Loading checkpoint from ../models/bert_classifier/model_step_1000.pt
Namespace(accum_count=1, alpha=0.95, batch_size=30, beam_size=5, bert_data_path='../small_data/cnndm', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0], label_smoothing=0.1, large=False, load_from_extractive='', log_file='../logs/ext_val_bert_cnndm', lr=1, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=200, max_pos=512, max_tgt_len=140, min_length=50, mode='validate', model_path='../models/bert_classifier', optim='adam', param_init=0, param_init_glorot=True, recall_eval=

[2019-11-09 15:09:02,262 INFO] Rouges at step 1000 
>> ROUGE-F(1/2/3/l): 39.38/17.08/35.79
ROUGE-R(1/2/3/l): 50.67/22.06/46.01

[2019-11-09 15:09:02,262 INFO] Validation xent: 6.18543 at step 1000
^C
Traceback (most recent call last):
  File "train.py", line 148, in <module>
    validate_ext(args, device_id)
  File "/Users/adas1/Aditi/personal/school/266/project/PreSumm/src/train_extractive.py", line 140, in validate_ext
    cp_files = sorted(glob.glob(os.path.join(args.model_path, 'model_step_*.pt')))
  File "/Users/adas1/anaconda3/lib/python3.7/glob.py", line 20, in glob
    return list(iglob(pathname, recursive=recursive))
  File "/Users/adas1/anaconda3/lib/python3.7/glob.py", line 72, in _iglob
    for name in glob_in_dir(dirname, basename, dironly):
  File "/Users/adas1/anaconda3/lib/python3.7/glob.py", line 80, in _glob1
    names = list(_iterdir(dirname, dironly))
  File "/Users/adas1/anaconda3/lib/python3.7/glob.py", line 122, in _iterdir
    for entry in it:
KeyboardInterrupt


## CNN_DM abstractive from Pretrained BERT but finetuned for BertSUM
Here you have to train extractive model first and pass that model to the abstractive model training

In [10]:
%cd PreSumm/src
!python train.py  -task abs -mode train -bert_data_path ../small_data/cnndm -dec_dropout 0.2  -model_path ../models/bert_classifier  -sep_optim true -lr_bert 0.002 -lr_dec 0.2 -save_checkpoint_steps 100 -batch_size 140 -train_steps 100 -report_every 100 -accum_count 5 -use_bert_emb true -use_interval true -warmup_steps_bert 200 -warmup_steps_dec 10000 -max_pos 512 -visible_gpus -1 -log_file ../logs/abs_bert_cnndm  -load_from_extractive ../models/bert_classifier/model_step_1000.pt


[Errno 2] No such file or directory: 'PreSumm/src'
/Users/adas1/Aditi/personal/school/266/project/PreSumm/src
[2019-11-09 19:45:19,399 INFO] Namespace(accum_count=5, alpha=0.6, batch_size=140, beam_size=5, bert_data_path='../small_data/cnndm', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0], label_smoothing=0.1, large=False, load_from_extractive='../models/bert_classifier/model_step_1000.pt', log_file='../logs/abs_bert_cnndm', lr=1, lr_bert=0.002, lr_dec=0.2, max_grad_norm=0, max_length=150, max_pos=512, max_tgt_len=140, min_length=15, mode='train', model_path='../models/bert_classifier', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=100, report_ro

          (layer_norm): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (dropout_1): Dropout(p=0.2, inplace=False)
          (dropout_2): Dropout(p=0.2, inplace=False)
        )
        (layer_norm_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (layer_norm_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (drop): Dropout(p=0.2, inplace=False)
      )
    )
    (layer_norm): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
  )
  (generator): Sequential(
    (0): Linear(in_features=768, out_features=30522, bias=True)
    (1): LogSoftmax()
  )
)
[2019-11-09 19:45:25,574 INFO] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ../temp/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
gpu_rank 0
[2019-11-09 19:45:25,669 INFO] * number of parameters: 180222522
[2019-11-09 19:45:25,669 INFO] Start traini

### Test with Rouge Score

In [11]:
%cd PreSumm/src
!python train.py -task abs -mode test -test_from ../models/bert_classifier/model_step_100.pt  -batch_size 30 -test_batch_size 500 -bert_data_path ../small_data/cnndm -log_file ../logs/val_abs_bert_cnndm -model_path ../models/bert_classifier -sep_optim true -use_interval true -visible_gpus -1 -max_pos 512 -max_length 200 -alpha 0.95 -min_length 50 -result_path ../logs/abs_tes_bert_cnndm

[Errno 2] No such file or directory: 'PreSumm/src'
/Users/adas1/Aditi/personal/school/266/project/PreSumm/src
[2019-11-09 20:26:53,744 INFO] Loading checkpoint from ../models/bert_classifier/model_step_100.pt
Namespace(accum_count=1, alpha=0.95, batch_size=30, beam_size=5, bert_data_path='../small_data/cnndm', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0], label_smoothing=0.1, large=False, load_from_extractive='', log_file='../logs/val_abs_bert_cnndm', lr=1, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=200, max_pos=512, max_tgt_len=140, min_length=50, mode='test', model_path='../models/bert_classifier', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False

## Run Transformer Summarizer

In [24]:
%cd /Users/adas1/Aditi/personal/school/266/project/transformers/examples
!export DATA_PATH=/Users/adas1/Aditi/personal/school/266/project/PreSumm/small_data

!python run_summarization_finetuning.py --output_dir=../output \
    --model_type=bert \
    --model_name_or_path=bert-base-cased \
    --do_train True \
    --data_dir=$DATA_PATH 

/Users/adas1/Aditi/personal/school/266/project/transformers/examples
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt not found in cache or force_download set to True, downloading to /var/folders/7m/bpr251m543ndyscyxrt1_m0r0000gn/T/tmpgvn1p05d
100%|████████████████████████████████| 213450/213450 [00:00<00:00, 831803.43B/s]
INFO:transformers.file_utils:copying /var/folders/7m/bpr251m543ndyscyxrt1_m0r0000gn/T/tmpgvn1p05d to cache at /Users/adas1/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:transformers.file_utils:creating metadata file for /Users/adas1/.cache/torch/transformers/5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:transformers.file_utils:removing temp file /var/folders/7m/bpr251m543ndyscyxrt1_m0r0000gn/T/tmpgvn1p05d
INFO